In [ ]:
import mujoco
import numpy as np
import pinocchio as pin

# from robot_descriptions.skydio_x2_description import URDF_PATH
# from robot_descriptions.skydio_x2_mj_description import MJCF_PATH
from robot_descriptions.z1_description import URDF_PATH
from robot_descriptions.z1_mj_description import MJCF_PATH

from mujoco_sysid import parameters, regressors
from mujoco_sysid.utils import muj2pin

In [ ]:
mjmodel = mujoco.MjModel.from_xml_path(MJCF_PATH)
# enable energy
mjmodel.opt.enableflags |= mujoco.mjtEnableBit.mjENBL_ENERGY
# disable friction, contact and limits
mjmodel.opt.disableflags |= (
    mujoco.mjtDisableBit.mjDSBL_CONTACT | mujoco.mjtDisableBit.mjDSBL_FRICTIONLOSS | mujoco.mjtDisableBit.mjDSBL_LIMIT
)
mjdata = mujoco.MjData(mjmodel)

pinmodel = pin.buildModelFromUrdf(URDF_PATH)
pindata = pin.Data(pinmodel)

In [ ]:
# np.random.seed(10)

In [ ]:
q, v = np.random.randn(mjmodel.nq), np.zeros(mjmodel.nv)


pinq, pinv = muj2pin(q, v)

In [ ]:
mjdata.qpos[:] = q.copy()
mjdata.qvel[:] = v.copy()

mujoco.mj_step(mjmodel, mjdata)

mj_en = mjdata.energy.copy()
mj_en[0] += regressors.potential_energy_bias(mjmodel)

mj_en, np.sum(mj_en)

In [ ]:
(
    np.sum(
        [
            pin.computePotentialEnergy(pinmodel, pindata, pinq),
            pin.computeKineticEnergy(pinmodel, pindata, pinq, pinv),
        ]
    ),
    [
        pin.computePotentialEnergy(pinmodel, pindata, pinq),
        pin.computeKineticEnergy(pinmodel, pindata, pinq, pinv),
    ],
)

In [ ]:
theta = np.concatenate([parameters.get_dynamic_parameters(mjmodel, i) for i in mjmodel.jnt_bodyid])

theta.shape, theta[:10]

In [ ]:
params = []

for i in range(len(pinmodel.inertias) - 1):
    params.extend(pinmodel.inertias[i + 1].toDynamicParameters())

    last_params = np.array(params[-10:])
    last_theta = theta[i * 10 : (i + 1) * 10]

    # mass should match
    assert np.isclose(last_params[0], last_theta[0])

    # lever arm should match
    assert np.allclose(last_params[1:4], last_theta[1:4])

    print(f"for body {i} norm of difference is {np.linalg.norm(last_params - last_theta)}")

params = np.array(params)

params.shape, np.linalg.norm(params - theta)

In [ ]:
reg_en = regressors.mj_energyRegressor(mjmodel, mjdata)[2]

reg_en @ theta